In [ ]:
from utils.processor import AudioProcessor
from utils.dataset_utils import get_concat_dataset
from utils.config_loader import load_config
from torchvision.transforms import Compose
import matplotlib.pyplot as plt
import librosa.display
import random

In [ ]:
config = load_config("./config/augment_test.yaml")
log_mel_conf = config.data.audio.log_mel_conf
ori_datasets = get_concat_dataset(
    [""], "train"
)
ori_processor = AudioProcessor(None, None, log_mel_conf, None, None, None)
ori_func = Compose([ori_processor.raw_to_logmelspect,ori_processor.output_transpose])
ori_datasets.set_transform(ori_func)
train_datasets = get_concat_dataset(
    [""], "train"
)

In [ ]:
normalize = None

# speed change augmentation
# speed_aug_conf = config.data.audio.speed_aug_conf
speed_aug_conf = None

# time or frequency masking (something to 0)
# spec_aug_conf = config.data.audio.spec_aug_conf
spec_aug_conf = None

# specific time copy and random point override
# spec_sub_conf = config.data.audio.spec_sub_conf
spec_sub_conf = None

# random sequence time section delete(trim)
spec_trim_conf = config.data.audio.spec_trim_conf
# spec_trim_conf = None

audio_processor = AudioProcessor(
    speed_aug_conf,
    normalize,
    log_mel_conf,
    spec_aug_conf,
    spec_sub_conf,
    spec_trim_conf,
)
training_get_func = Compose(
    [
        audio_processor.raw_audio_preprocess,
        audio_processor.raw_to_logmelspect,
        audio_processor.spectrogram_preprocess,
        audio_processor.output_transpose,
    ]
)
train_datasets.set_transform(training_get_func)

In [ ]:
random_idx = random.randint(0, len(train_datasets)-1)
fig, ax = plt.subplots()
img = librosa.display.specshow(ori_datasets[320000]["input_values"].T.numpy(), x_axis='time', y_axis='linear', ax=ax)
ax.set(title='ORIGINAL!!!!!')
fig.colorbar(img, ax=ax, format="%+2.f dB")

In [ ]:
random_idx = random.randint(0, len(train_datasets)-1)
fig, ax = plt.subplots()
temp = train_datasets[320000]["input_values"].T.numpy()
img = librosa.display.specshow(temp, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='AUG FINAL!!!!')
fig.colorbar(img, ax=ax, format="%+2.f dB")

In [ ]:
import numpy as np
win_length = int(np.ceil(log_mel_conf.sample_rate * log_mel_conf.window_size_sec))
n_fft = win_length
hop_length = int(log_mel_conf.sample_rate * log_mel_conf.window_stride_sec)
res = librosa.feature.inverse.mel_to_audio(temp, 
                                           sr=log_mel_conf.sample_rate, 
                                           n_fft=n_fft, 
                                           hop_length=hop_length, 
                                           win_length=win_length, 
                                           window='hamming',
                                           center=True, 
                                           pad_mode='reflect', 
                                           power=2.0, 
                                           n_iter=32)
# ori = librosa.feature.inverse.mel_to_audio(ori_datasets[320000]["input_values"].T.numpy(), 
#                                            sr=log_mel_conf.sample_rate, 
#                                            n_fft=n_fft, 
#                                            hop_length=hop_length, 
#                                            win_length=win_length, 
#                                            window='hamming',
#                                            center=True, 
#                                            pad_mode='reflect', 
#                                            power=2.0, 
#                                            n_iter=32)
import soundfile as sf
sf.write("./test1.wav", res, log_mel_conf.sample_rate)
# sf.write("./ori1.wav", ori, log_mel_conf.sample_rate)

In [ ]:
sf.write("./ori_ori.wav",np.array(ori_datasets[320000]["input_values"]), 16000)